In [1]:
import sys
import time

sys.path.append("..")
import pandas as pd
import solv_fzcp as sfzcp
import uvarprob as uvpr

In [2]:
test_file = "../test_casado2.csv"


In [3]:
def read_problems(fname):
    data = pd.read_csv(fname, index_col='name', comment='#')
    return data


df = read_problems(test_file)
df

,objective,a,b,min_f,min_x
name,,,,,
16,sqrt(x)*(sin(x))^2,0.2,7.0,0,0
17,cos(x)-sin(5.0*x)+1,0.2,7.0,0,0
18,-x-sin(3.0*x)+1.6,0.2,7.0,0,0
19,cos(x)+2.0*cos(2.0*x)*exp(-x),0.2,7.0,0,0
20,-(sin(2*x+1)+2*sin(3*x+2)+3*sin(4*x+3)+4*sin(...,-10.0,10.0,0,0
21,log(3.0*x)*log(2*x)-1,0.1,7.0,0,0
22,-cos(2.0*x)-cos(3*x)-cos(4*x)-cos(5*x)-cos(6*x),-10.0,10.0,0,0
23,-exp(-x)*sin(2*pi*x)+0.5,0.0,4.0,0,0
24,(x^2-5*x+6)/(x^2+1)-0.5,-5.0,5.0,0,0


In [4]:
points_db = {}
psl_lipint_points_list = []
psl_lip_points_list = []
psqe_lipint_points_list = []
psqe_lip_points_list = []
bnb2_lipint_points_list = []
bnb2_lip_points_list = []

In [5]:
def log_point(x, points_list):
    points_list.append(x)

In [6]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    # #
    # prob = uvpr.UniVarProblem(test.Index, test.objective, test.a, test.b, test.min_f, test.min_x,
    #                           lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']))
    # period = 0
    # T1 = time.perf_counter()
    # print('Piyavskii, Lipschitz interval:',
    #       sfzcp.correctly(prob, symm=False, epsilon=1e-10, global_lipschitz_interval=False, estimator=1, reduction=1,
    #                        period_comp_lip=period))
    # T2 = time.perf_counter()
    # print('time=', (T2 - T1)*1000, 'period:', period)
    #
    iaf.set_log_taylor_terms_number(19)
    iaf.set_cos_taylor_terms_number(19)
    iaf.set_sin_taylor_terms_number(19)
    iaf.set_exp_taylor_terms_number(19)
    ia.set_precision(19)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

16
PSQE: TestResult(nsteps=15, first_crossing_zero_point=[[3.141590130066791268, 3.141595198923878073]])
time= 221.42479219473898 period: 0
Linear: TestResult(nsteps=45, first_crossing_zero_point=[[3.141589063786179310, 3.141594374635865331], [6.283177606031934048, 6.283187424529999104]])
time= 250.91766705736518 period: 0
17
PSQE: TestResult(nsteps=8, first_crossing_zero_point=[[1.570796320167844205, 1.570796327322299965]])
time= 83.85624992661178 period: 0
Linear: TestResult(nsteps=12, first_crossing_zero_point=[[1.570796326567744982, 1.570796391420169378]])
time= 86.15487487986684 period: 0
18
PSQE: TestResult(nsteps=7, first_crossing_zero_point=[[1.968571239642353546, 1.968571239662884470]])
time= 63.22666699998081 period: 0
Linear: TestResult(nsteps=9, first_crossing_zero_point=[[1.968571239658404011, 1.968571239662585282]])
time= 46.85124987736344 period: 0
19
PSQE: TestResult(nsteps=5, first_crossing_zero_point=[[1.140713605989841417, 1.140713619775323432]])
time= 188.5143748950

In [10]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(19)
    iaf.set_cos_taylor_terms_number(19)
    iaf.set_sin_taylor_terms_number(19)
    iaf.set_exp_taylor_terms_number(19)
    ia.set_precision(19)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    print('PSQE:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=2, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

    iaf.set_log_taylor_terms_number(9)
    iaf.set_cos_taylor_terms_number(9)
    iaf.set_sin_taylor_terms_number(9)
    iaf.set_exp_taylor_terms_number(9)
    ia.set_precision(9)
    T1 = time.perf_counter()
    print('Linear:',
          sfzcp.correctly(prob, symm=False, epsilon=1e-5, global_lipschitz_interval=False, estimator=1, reduction=1,
                          adaptive=False))
    T2 = time.perf_counter()
    print('time=', (T2 - T1) * 1000, 'period:', period)

1
PSQE: TestResult(nsteps=3, first_crossing_zero_point=[[3.011688528945636903, 3.011690771062112775]])
time= 13.515874976292253 period: 0
Linear: TestResult(nsteps=5, first_crossing_zero_point=[[3.01168617, 3.01169087]])
time= 6.573165999725461 period: 0
3
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[1.174796171291449297, 1.174796171291492291]])
time= 82.15316606219858 period: 0
Linear: TestResult(nsteps=8, first_crossing_zero_point=[[1.17479606, 1.17479644]])
time= 18.98870791774243 period: 0
4
PSQE: TestResult(nsteps=6, first_crossing_zero_point=[[2.960913794435176517, 2.960913794442501751]])
time= 156.35225002188236 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[2.96091375, 2.96091383]])
time= 53.09837497770786 period: 0
5
PSQE: TestResult(nsteps=9, first_crossing_zero_point=[[0.8209239701115810884, 0.8209239701115810941]])
time= 53.084708051756024 period: 0
Linear: TestResult(nsteps=10, first_crossing_zero_point=[[0.820923955, 0.820924025]])
time= 14.

In [6]:
import ia_math_fun as iaf
import interval_arithmetics as ia

for test in df.itertuples():
    print(test.Index)
    points_db[test.Index] = {'psl_lipint_points_list': [], 'psl_lip_points_list': [], 'psqe_lipint_points_list': [],
                             'psqe_lip_points_list': [], 'bnb2_pslint_points_list': [], 'bnb2_psl_points_list': [],
                             'bnb2_lipint_points_list': [],
                             'bnb2_lip_points_list': []}

    iaf.set_log_taylor_terms_number(11)
    iaf.set_cos_taylor_terms_number(11)
    iaf.set_sin_taylor_terms_number(11)
    iaf.set_exp_taylor_terms_number(11)
    ia.set_precision(11)
    prob = uvpr.UniVarProblem(name=test.Index, objective=test.objective, a=test.a, b=test.b, min_f=test.min_f,
                              min_x=test.min_x,
                              logger=lambda x: log_point(x, points_db[test.Index]['bnb2_pslint_points_list']),
                              preproc=True, correctly=True)
    period = 0
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=True, estimator=2, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Quadratic, global lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=True, estimator=1, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Linear, global lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)

    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=2, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Quadratic, local lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)
    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=1, reduction=1,
                            adaptive=False).nsteps
    T2 = time.perf_counter()
    print('Linear, local lipschitz')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)

    T1 = time.perf_counter()
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=False, estimator=2, reduction=1,
                            adaptive=True).nsteps
    T2 = time.perf_counter()
    print('Quadratic, adaptive lipschitz v1')
    print('steps=', steps, 'time=', (T2 - T1) * 1000)


16
[3.1415141527, 3.1415141536]
[3.1415642907, 3.1415642916]
err
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/wuzhongqi/miniforge3/envs/py39/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/x6/jmxw5p4x1c30zhxg3hykkdrw0000gn/T/ipykernel_12634/795247205.py", line 22, in <module>
    steps = sfzcp.correctly(prob, symm=False, epsilon=1e-6, global_lipschitz_interval=True, estimator=2, reduction=1,
  File "/Users/wuzhongqi/PycharmProjects/univarsolver/FZCP/solv_fzcp.py", line 122, in correctly
    steps = bnb.bnb_fzcp(sl, cnt, psp)
  File "/Users/wuzhongqi/PycharmProjects/univarsolver/bnb.py", line 37, in bnb_fzcp
    new_subs = processor.fzcp_process(s)
  File "/Users/wuzhongqi/PycharmProjects/univarsolver/FZCP/processor_correctly.py", line 152, in fzcp_process
    right_end = lower_estimator.get_right_end_under_bound()
  File "/Users/wuzhongqi/PycharmProjects/univarsolver/FZCP/psqe_bounds_correctly.py", line 493, in get_right_end_under_b